In [1]:
# Importing Dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Avarua,CK,2021-09-16 08:10:38,-21.2078,-159.7750,75.25,78,40,8.05
1,1,Chapais,CA,2021-09-16 08:20:35,49.7834,-74.8492,46.18,98,21,4.50
2,2,Jamestown,US,2021-09-16 08:11:21,42.0970,-79.2353,57.09,99,90,1.99
3,3,Khatanga,RU,2021-09-16 08:18:29,71.9667,102.5000,35.38,94,100,12.55
4,4,Nikolskoye,RU,2021-09-16 08:13:35,59.7035,30.7861,47.01,82,100,3.65


In [3]:
# Configure gmaps to use API key
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False,
                                max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of windiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask customer temperature range
#min_temp = float(input("What is the lowest temperature you would like for your trip? "))
#max_temp = float(input("What is the highest temperature you would like for your trip? "))
min_temp = 75
max_temp = 90

In [9]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & 
                                    (city_data_df['Max Temp'] <= max_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Avarua,CK,2021-09-16 08:10:38,-21.2078,-159.7750,75.25,78,40,8.05
17,17,Oktyabrskoye,RU,2021-09-16 08:20:46,43.0645,44.7417,75.34,41,0,2.24
18,18,Vaini,TO,2021-09-16 08:13:32,-21.2000,-175.2000,75.36,94,20,6.91
25,25,Sao Filipe,CV,2021-09-16 08:15:19,14.8961,-24.4956,79.65,81,35,9.13
28,28,Mitsamiouli,KM,2021-09-16 08:20:51,-11.3847,43.2844,88.09,48,40,8.05


In [10]:
preferred_cities_df.count()

City_ID       199
City          199
Country       197
Date          199
Lat           199
Lng           199
Max Temp      199
Humidity      199
Cloudiness    199
Wind Speed    199
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Avarua,CK,75.25,-21.2078,-159.7750,
17,Oktyabrskoye,RU,75.34,43.0645,44.7417,
18,Vaini,TO,75.36,-21.2000,-175.2000,
25,Sao Filipe,CV,79.65,14.8961,-24.4956,
28,Mitsamiouli,KM,88.09,-11.3847,43.2844,
32,Bambous Virieux,MU,76.21,-20.3428,57.7575,
33,Tautira,PF,77.02,-17.7333,-149.1500,
35,Ajdabiya,LY,79.72,30.7554,20.2263,
38,Cockburn Town,TC,82.62,21.4612,-71.1419,
39,Coro,VE,78.91,11.4045,-69.6734,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        
hotel_df.head

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


<bound method NDFrame.head of               City Country  Max Temp      Lat       Lng  \
0           Avarua      CK     75.25 -21.2078 -159.7750   
17    Oktyabrskoye      RU     75.34  43.0645   44.7417   
18           Vaini      TO     75.36 -21.2000 -175.2000   
25      Sao Filipe      CV     79.65  14.8961  -24.4956   
28     Mitsamiouli      KM     88.09 -11.3847   43.2844   
..             ...     ...       ...      ...       ...   
551  San Policarpo      PH     75.92  12.1791  125.5072   
552      Prokuplje      RS     78.57  43.2342   21.5881   
553      Boa Vista      BR     82.38   2.8197  -60.6733   
555         Bombay      IN     85.98  19.0144   72.8479   
558         Sinjar      IQ     85.98  36.3209   41.8754   

                             Hotel Name  
0                          Paradise Inn  
17                  Prolesok Mini-hotel  
18                  Keleti Beach Resort  
25                        Hotel Xaguate  
28                            Foyer ADM  
..       

In [14]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


In [16]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))